In [ ]:
from estimator import find_deg
import math

def print_instances(params):
    print("{:<12} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format(
        "log2 q", "log2 b", "degree", "secret size", "num poly", "block size (B)", "hash size (B)"))
    for param in params:
        print("{:<12} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format(
            param.log2_modulus,
            param.log2_norm,
            param.degree,
            param.degree * param.num_poly,
            param.num_poly,
            param.block_size_bytes,
            param.hash_size_bytes,
        ))

def find_all_instances(log2_q, secret_size_bounds):
    params = []
    num_limbs = 2
    # Initialization of beta at q, but could be any large number
    log2_beta = log2_q
    for num_limbs in range(log2_q, 1, -1):
        # print("Working on #Limbs {}".format(num_limbs))
        # This value of beta
        if math.ceil(log2_q / num_limbs) == log2_beta:
            # Previous iteration could do smaller limbs
            continue
        log2_beta = math.ceil(log2_q / num_limbs)
        # print(log2_beta)
        # Estimates for all the `m` in range
        for m in range(secret_size_bounds[0], secret_size_bounds[1]):
            # print("\tm = {}".format(m))
            params.append(find_deg(log2_q, log2_beta, 2**m, margin_bkz=16, margin_cpw=16))
    return params

params = find_all_instances(64, (10, 25))
print_instances(params)


In [24]:
from bkz import bkz_bit_security, estimate_bkz_security_for_m, PESSIMISTIC, HEURISTIC, SVP_ORACLES, log2gamma
from combinatorials import cpw_bit_security, cpw_bit_security_pessimistic, estimate_cpw_with_hnf_and_parameters, estimate_cpw_with_parameters
import math


old_params = [\
    ["A1", 64, 5, 32],\
    ["A2", 64, 7, 64],\
    ["A3", 64, 11, 128],\
    ["A4", 64, 16, 256],\
    ["A5", 64, 22, 512],\
    ["A6", 64, 32, 1024],\
    ["B1", 254, 3, 2],\
    ["B2", 254, 4, 4],\
    ["B3", 254, 5, 8],\
    ["B4", 254, 7, 16],\
    ["B5", 254, 11, 32],\
    ["B6", 254, 16, 64],\
    ["B7", 254, 24, 128],\
    ["B8", 254, 32, 256],\
    ["B9", 254, 51, 512],\
]

new_params = [\
    ["A1", 64, 2, 32],\
    ["A2", 64, 4, 64],\
    ["A3", 64, 6, 128],\
    ["A4", 64, 10, 256],\
    ["A5", 64, 16, 512],\
    ["A6", 64, 22, 1024],\
    ["B3", 254, 1, 3],
    ["B3", 254, 2, 7],
    ["B4", 254, 4, 16],\
    ["B5", 254, 6, 32],\
    ["B6", 254, 10, 64],\
    ["B7", 254, 16, 128],\
    ["B8", 254, 24, 256],\
    ["B9", 254, 32, 512],\
]

hundreds = [\
    ["A1", 256, 1, 3],
    ["A1", 256, 2, 5],
    ["A1", 256, 3, 8],
    ["A1", 256, 4, 11],
    ["A1", 256, 5, 16],
    ["A1", 256, 6, 20],
    ["A1", 256, 16, 98],
    
]

def direct_svp_cost(n, logbeta, logq):
    m = math.ceil(logq/logbeta - 1)
    dim = (m+1)*n
    # Attack using direct SVP attack
    return SVP_ORACLES["0.292 k + 16.4"](dim)

def estimate_set_of_params(params):
    print("|{:<12}|{:<12}|{:<12}|{:<12}|{:<12}|{:<12}|{:<12}|{:<12}".format("$log_2(q)$", "$log_2(\beta)$", "$n$", "direct SVP attack", "BKZ attack", "CPW attack", "#limbs", "pi"))
    for param in params:
        logq, logbeta, n = param[1], param[2], param[3]
        svp_cost = direct_svp_cost(n=n, logbeta=logbeta, logq=logq)
        bkz_csot = bkz_bit_security(log2_beta=logbeta, log2_q=logq, n=n)
        cpw_cost = cpw_bit_security(log2_beta=logbeta, log2_q=logq, n=n, m_0=2**21)
        numlimbs = math.ceil(256 / logbeta)
        prod = numlimbs*n
        print("|{:<12}|{:<12}|{:<12}|{:<12}|{:<12}|{:<12}|{:<12}".format(logq, logbeta, n, round(svp_cost, ndigits=2), round(bkz_csot, ndigits=2), round(cpw_cost, ndigits=2), numlimbs), round(prod, ndigits=2))

#estimate_set_of_params(old_params)
#estimate_set_of_params(new_params)
estimate_set_of_params(hundreds)

|$log_2(q)$  |$log_2(eta)$|$n$         |direct SVP attack|BKZ attack  |CPW attack  |#limbs      |pi          
|256         |1           |3           |240.66      |149.92      |227.0       |256          768
|256         |2           |5           |203.28      |113.25      |253.0       |128          640
|256         |3           |8           |217.3       |107.78      |263.09      |86           688
|256         |4           |11          |221.97      |99.75       |267.0       |64           704
|256         |5           |16          |259.34      |105.59      |272.0       |52           832
|256         |6           |20          |267.52      |100.44      |501.0       |43           860
|256         |16          |98          |474.26      |100.73      |1941.85     |16           1568


In [18]:
from math import sqrt, pi, e

params = [\
    ["A1", 64, 5, 32],\
    ["A2", 64, 7, 64],\
    ["A3", 64, 11, 128],\
    ["A4", 64, 16, 256],\
    ["A5", 64, 22, 512],\
    ["A6", 64, 32, 1024],\
    ["B1", 254, 3, 2],    
    ["B2", 254, 4, 4],
    ["B3", 254, 5, 8],
    ["B4", 254, 7, 16],\
    ["B5", 254, 11, 32],\
    ["B6", 254, 16, 64],\
    ["B7", 254, 24, 128],\
    ["B8", 254, 32, 256],\
    ["B9", 254, 51, 512],\
]


def log_svp_cost(dim):
    return 0.292 * dim + 16.4


def find_m_prime(n, beta, q):
    for m in range(1,10000):
        if beta > q**(1/(m+1)):
            return m

for param in params:
    m = find_m_prime(param[3], 2**(param[2]), 2**param[1])
    dim = round((m + 1) * param[3])
    print (param[0], m, dim, log_svp_cost(dim))

A1 12 416 137.87199999999999
A2 9 640 203.28
A3 5 768 240.65599999999998
A4 4 1280 390.15999999999997
A5 2 1536 464.9119999999999
A6 2 3072 913.4239999999999
B1 84 170 66.03999999999999
B2 63 256 91.15199999999999
B3 50 408 135.536
B4 36 592 189.26399999999998
B5 23 768 240.65599999999998
B6 15 1024 315.40799999999996
B7 10 1408 427.53599999999994
B8 7 2048 614.4159999999999
B9 4 2560 763.92


In [1]:
from bkz import bkz_log_root_hermite, log2_volume_unit_ball
m = 1000

bkz_log_root_hermite(m, m), -1/(m*m)*log2_volume_unit_ball(m)

(0.0029401331628127136, 0.0029416054378066108)

In [35]:
# bkz_bit_security(64, 32, 5, 2**20, norm_equivalence_strategy=HEURISTIC)
# >> 164.91
# bkz_bit_security(64, 32, 5, 2**20, norm_equivalence_strategy=PESSIMISTIC)
# >> 111.66

# bkz_bit_security(64, 64, 7, 2**20, norm_equivalence_strategy=HEURISTIC)
# >> 170.55
# bkz_bit_security(64, 64, 7, 2**20, norm_equivalence_strategy=PESSIMISTIC)
# >> 133.37

# bkz_bit_security(64, 128, 11, 2**20, norm_equivalence_strategy=HEURISTIC)
# >> 144.82
# bkz_bit_security(64, 128, 11, 2**20, norm_equivalence_strategy=PESSIMISTIC)
# >> 125.09

# bkz_bit_security(64, 256, 16, 2**20, norm_equivalence_strategy=HEURISTIC)
# >> 148.39
# bkz_bit_security(64, 256, 16, 2**20, norm_equivalence_strategy=PESSIMISTIC)
# >> 134.05

# bkz_bit_security(64, 512, 22, 2**20, norm_equivalence_strategy=HEURISTIC)
# >> 170.41
# bkz_bit_security(64, 512, 22, 2**20, norm_equivalence_strategy=PESSIMISTIC)
# >> 157.99

# bkz_bit_security(64, 1024, 32, 2**20, norm_equivalence_strategy=HEURISTIC)
# >> 171.83
# bkz_bit_security(64, 1024, 32, 2**20, norm_equivalence_strategy=PESSIMISTIC)
# >> 162.88

n repetitions  2409.602766449696 number of solver rounds 0.018311197803364987 oracle costs 244.79778555257693
n repetitions  2650.5630430946658 number of solver rounds -1.076805130217482 oracle costs 191.80160960217881
n repetitions  2915.6193474041324 number of solver rounds -1.7856019045561862 oracle costs 168.98884597456325
n repetitions  3207.181282144546 number of solver rounds -2.286693899349032 oracle costs 158.31380235227883
n repetitions  3527.8994103590007 number of solver rounds -2.653329713472257 oracle costs 154.205560824406
n repetitions  3880.689351394901 number of solver rounds -2.9253022332605827 oracle costs 154.33966950978999
n repetitions  4268.758286534392 number of solver rounds -3.1274139450695646 oracle costs 157.60123112687685
n repetitions  4695.634115187831 number of solver rounds -3.276417102166502 oracle costs 163.42423022694493
n repetitions  5165.197526706615 number of solver rounds -3.3842222588843294 oracle costs 171.52817521354334
n repetitions  5681.7

162.8833290746413

0.005880267317979214